# LORE - Repository Analysis Tool

This notebook sets up and runs the LORE application in a remote server environment.

## 1. Setup Environment

First, let's create a virtual environment and install the required dependencies:

In [ ]:
!python -m venv .venv
!source .venv/bin/activate
!pip install -r requirements-jupyter.txt

## 2. Set Environment Variables

Set up your API keys and configuration:

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Verify environment variables are loaded
required_vars = ['LLAMA_API_KEY', 'LLAMA_API_BASE']
missing_vars = [var for var in required_vars if not os.getenv(var)]
if missing_vars:
    raise ValueError(f"Missing required environment variables: {', '.join(missing_vars)}")

## 3. Import Required Modules

In [ ]:
import sys
from pathlib import Path

# Add the project root to Python path
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from lore.llm.llama_client import LlamaClient
from lore.analysis.analyzer import RepositoryAnalyzer
from lore.ui.chat_interface import RepoChat
from lore.utils.repo_utils import clone_github_repo
from lore.ingestion.git_extractor import GitExtractor

## 4. Initialize Components and Analyze Repository

In [ ]:
# Initialize LlamaClient
llm_client = LlamaClient()

# Initialize Repository Analyzer
analyzer = RepositoryAnalyzer(llm_client)

# Clone repository
repo_url = "https://github.com/rithwik-01/LlamaConHackathon"  # Replace with your repo URL
repo_path = clone_github_repo(repo_url)
print(f"Repository cloned to: {repo_path}")

# Create GitExtractor
git_extractor = GitExtractor(repo_path)

# Prepare repository context
print("Preparing repository context...")
repo_context = analyzer.prepare_repository_context(git_extractor)

# Analyze repository
print("Analyzing repository...")
analysis_result = analyzer.analyze_repository(
    context=repo_context,
    task="analyze_architecture",
    model="llama-2-70b-chat"
)
print("Analysis complete!")

# Initialize chat interface
chat = RepoChat(llm_client, analyzer, repo_context)
print("Chat interface ready!")

## 5. Chat Interface

Use this cell to interact with the repository:

In [ ]:
def ask_question(question: str):
    """Ask a question about the repository"""
    print(f"Q: {question}\n")
    try:
        response = chat.chat(question)
        print(f"A: {response}\n")
        return response
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

# Example usage:
question = "What are the main components of this repository?"
ask_question(question)